### Démarrage de tensorboard et imports principaux

In [1]:
import os

os.environ['TF_DISABLE_MKL'] = '1'

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
# Agrandir le notebook ?
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

# Démarrage de tensorboard pour notebook
%load_ext tensorboard

import sys
from matplotlib import pyplot
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import *
from tensorflow.keras.models import *
from tensorflow.keras import regularizers
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.activations import *
import numpy as np
import datetime
import os
import pandas as pd
from PIL import Image
import shutil  
from math import ceil, floor


from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.framework import ops #pour tenter de reset tensorboard, sans grand succès
from sklearn.model_selection import train_test_split as tts

ops.reset_default_graph()

### Hyper paramètres


In [3]:
# Parametres de verification : 

list_indiv_id = ['1', '2']
list_epochs = [50, 50]
list_batch_size = [320, 350]
list_blocks_size = [2, 2]
list_blocks_nb = [16, 16]
list_l1 = [0, 0]
list_l2 = [0, 0]
list_batch_norm = [0, 0]
list_dropout = [0, 0]
list_filters_per_layers = [64, 64]
list_filters_double = [4, 3] #défini quand doubler cette valeur
list_filters_pool = [4, 3] #défini tout les combien de block est effectué une couche de pooling, ne peut être nul
list_activation = ['relu', 'relu']
list_kernel = [(3,3), (3,3)]
list_first_kernel = [(7,7), (7,7)]
list_padding = ['same', 'same']
list_max_or_avg_pool = ['avg', 'avg']
list_learning_r = [0.001, 0.001]
list_momentum = [0.9, 0.9]
list_optimizer = ['Adam', 'Adam']

current_time = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")
main_directory = os.getcwd() + "\\logs\\resnetsConv2d\\logs_" + current_time


import time
import json

# Fonctions pour préparer le dataset

In [4]:
with tf.device("/cpu:0"):
    DIR_TEST = 'D:/Projets/DL_4A_2nd/dataset/nybg2020/test/'
    DIR_TRAIN = 'D:/Projets/DL_4A_2nd/dataset/nybg2020/train/'
    META_DATA_TRAIN = DIR_TRAIN+'metadata.json'
    META_DATA_TEST = DIR_TEST+'metadata.json'
    
    start_time = time.time()
    
    with open(META_DATA_TRAIN, 'r') as json_file:
        data_train = json.load(json_file)
        for key in data_train:
            print("-",key, len(data_train[key]))
    print("================================")
    with open(META_DATA_TEST, 'r') as json_file:
        data_test = json.load(json_file)
        for key in data_test:
            print("-",key, len(data_test[key]))

- annotations 1030747
- categories 32094
- images 1030747
- info 6
- licenses 1
- regions 4
- images 138292
- info 6
- licenses 1


### TRAIN DATASET

In [5]:
with tf.device("/cpu:0"):    
    df_images = pd.DataFrame(data_train['images'])
    df_categories = pd.DataFrame(data_train['categories'])
    df_annotations = pd.DataFrame(data_train['annotations'])
    assert len(df_annotations) == len(df_images)
    df_images_annotations = pd.merge(df_images, df_annotations, left_on='id', right_on='image_id', how='right').drop('image_id', axis=1)
    df_images_annotations = df_images_annotations.sort_values(['category_id'])
    
    df_families = df_categories.drop('genus', axis=1).drop('id', axis=1).drop('name', axis=1).drop_duplicates().reset_index().drop('index', axis=1)
    df_families.head(3)
    
    df_images_infos = pd.merge(df_images_annotations, df_categories, left_on='category_id', right_on='id', how='right')\
                    .drop('height', axis=1).drop('id_x', axis=1).drop('license', axis=1).drop('width', axis=1)\
                    .drop('category_id', axis=1).drop('id_y', axis=1).drop('region_id', axis=1).drop('name', axis=1)

    df_images_infos = df_images_infos.rename(columns={"id":"category_id"})

    df_images_infos.sort_values(by='family', ascending=False)

### TEST DATASET

In [ ]:
with tf.device("/cpu:0"):
    df_images = pd.DataFrame(data_test['images']).drop('height', axis=1).drop('width', axis=1).drop('license', axis=1)
    df_images.head(3)

### SAVE DATA

In [ ]:
with tf.device("/cpu:0"):    
    df_images_infos.to_csv('D:/Projets/DL_4A_2nd/full_train_data.csv', index=False)
    df_images.to_csv('D:/Projets/DL_4A_2nd/full_test_data.csv', index=False)

### EXPLORE DATA / DATA GENERATOR

In [6]:
with tf.device("/cpu:0"):
    family = df_images_infos[['family', 'genus', 'category_id']].groupby(['family', 'genus']).count()
    display(family.describe())
    
    train_datagen = ImageDataGenerator(featurewise_center=False,
                                         featurewise_std_normalization=False,
                                         rotation_range=180,
                                         width_shift_range=0.1,
                                         height_shift_range=0.1,
                                         zoom_range=0.2)
    
    m = df_images_infos[['file_name', 'family', 'genus', 'category_id']]
    fam = m.family.unique().tolist()
    m.family = m.family.map(lambda x: fam.index(x))
    gen = m.genus.unique().tolist()
    m.genus = m.genus.map(lambda x: gen.index(x))
    display(m)

,category_id
count,3680.000000
mean,280.094565
std,763.668945
min,2.000000
25%,12.000000
50%,54.000000
75%,198.000000
max,14490.000000


,file_name,family,genus,category_id
0,images/000/00/626762.jpg,0,0,0
1,images/000/00/72077.jpg,0,0,0
2,images/000/00/818271.jpg,0,0,0
3,images/000/00/495523.jpg,0,0,0
4,images/000/00/437000.jpg,0,0,0
...,...,...,...,...
1030743,images/320/93/946506.jpg,41,3677,32093
1030744,images/320/93/1010199.jpg,41,3677,32093
1030745,images/320/93/35450.jpg,41,3677,32093
1030746,images/320/93/68259.jpg,41,3677,32093


# FORMAT DATA AND SELECT TRAIN/TEST

In [13]:
print("KILL THE NAN VALUE IN M TO PREVENT BUGS")
print(type(m))
na = m.file_name.isna()
keep = [x for x in range(m.shape[0]) if not na[x]]
m = m.iloc[keep]

train, verif = tts(m, test_size=0.2, shuffle=True, random_state=17)
train = train[:800000]
verif = verif[:200000]
shape = (64, 64, 3)

KILL THE NAN VALUE IN M TO PREVENT BUGS
<class 'pandas.core.frame.DataFrame'>


# Classe Python pour définir les individus

In [8]:
# Classe pour les convnets
class IndividuResnetConv2d:
    def __init__(self, indiv_id='1', epochs=10, batch_size=1, block_size=1, block_nb=1, l1=0, l2=0, batch_norm=0, dropout=0, filters_per_layers=64, filters_double=3, filters_pool=1, activation='relu', kernel=(3,3), first_kernel=(7,7), padding='same', max_or_avg_pool=0, learning_r=0.01, momentum=0.9, optimizer='SGD'):
        # Initialisation de nos variables
        self.time_fit = datetime.datetime.now()
        self.my_reguralizer = None
        
        if block_size < 1:
            self.block_size = 1
        else:
            self.block_size = block_size
            
        if block_nb < 2:
            self.block_nb = 1
        else:
            self.block_nb = block_nb
            
        self.loss = 0
        self.accuracy = 0
        self.indiv_id = indiv_id
        self.epochs = epochs
        self.batch_size = batch_size

        self.l1 = l1
        self.l2 = l2

        self.batch_norm = batch_norm
        self.dropout = dropout
        self.filters_per_layers = filters_per_layers
        
        # Création d'un variable qui va garder la valeur de filters_per_layers (elle changera dans le modèle)
        self.keep_filters_per_layers = filters_per_layers
        
        if filters_double < 2:
            self.filters_double = 2
        else : 
            self.filters_double = filters_double
            
        if filters_pool < 1:
            self.filters_pool = 1
        else : 
            self.filters_pool = filters_pool
        
        
        self.activation = activation
        self.kernel = kernel
        self.first_kernel = first_kernel
        self.padding = padding
        self.max_or_avg_pool = max_or_avg_pool
        self.learning_r = learning_r
        self.momentum = momentum
        self.optimizer = optimizer
        
        
        
    
    # ToString()
    def __str__(self):
        ma_liste = []
        ma_liste.append("indiv_id:{},\n ".format(self.indiv_id))
        ma_liste.append("epochs:{},\n ".format(self.epochs))
        ma_liste.append("batch_size:{},\n ".format(self.batch_size))
        ma_liste.append("block_size:{},\n ".format(self.block_size))
        ma_liste.append("block_nb:{},\n ".format(self.block_nb))
        ma_liste.append("l1:{},\n ".format(self.l1))
        ma_liste.append("l2:{},\n ".format(self.l2))
        ma_liste.append("batch_norm:{},\n ".format(self.batch_norm))
        ma_liste.append("dropout:{},\n ".format(self.dropout))
        ma_liste.append("filters_per_layers:{},\n ".format(self.filters_per_layers))
        ma_liste.append("filters_double:{},\n ".format(self.filters_double))
        ma_liste.append("filters_pool:{},\n ".format(self.filters_pool))
        ma_liste.append("activation:{},\n ".format(self.activation))
        ma_liste.append("kernel:\n ")
        ma_liste.append("{},\n ".format(self.kernel))
        ma_liste.append("first_kernel:\n ")
        ma_liste.append("{},\n ".format(self.first_kernel))
        ma_liste.append("padding:{},\n ".format(self.padding))
        ma_liste.append("max_or_avg_pool:{}\n".format(self.max_or_avg_pool))
        ma_liste.append("learning_r:{}\n".format(self.learning_r))
        ma_liste.append("momentum:{}\n".format(self.momentum))
        ma_liste.append("optimizer:{}\n".format(self.optimizer))
            
        return ma_liste
    
    
    
    
    # (Modele 2 conv + norm ? + pool) * X -> MLP -> softmax sortie 10 -> MODELE BLOC 2
    # D'autres modeles seront crees par la suite
    def create_model(self, main_directory, shape):
        start = datetime.datetime.now()
        
        # Choix d'un emplacement pour les logs
        log_dir=main_directory+"\\log_"+self.indiv_id+"\\tensorboard_data\\"
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
        print("log dir = ",log_dir)
        
        # l1 et l2
        if self.l1 > 0 and self.l2 > 0:
            self.my_regularizer = regularizers.l1_l2(l1=self.l1 / (self.block_nb*self.block_size), l2=self.l2 / (self.block_nb*self.block_size))
        if self.l1 > 0:
            self.my_regularizer = regularizers.l1(self.l1 / (self.block_nb*self.block_size))
        elif self.l2 > 0:
            self.my_regularizer = regularizers.l2(self.l2 / (self.block_nb*self.block_size))
        else:
            self.my_reguralizer = None
            
        

        # Faire toutes les convs nécessaires (conv * 2 + max pool)
        double_count = 0 # Var pour doubler les filtres
        
        input_layer = Input(shape)
        count = 1
        count2 = 1

        last_output = Conv2D(self.filters_per_layers, self.first_kernel, padding=self.padding, input_shape=(32, 32, 3), name='conv_'+str(self.filters_per_layers)+'_0',
                      kernel_regularizer=self.my_reguralizer)(input_layer)
        last_output = Activation(activation=self.activation, name="Activation_0")(last_output)

        if self.max_or_avg_pool == 'max':
            last_output = MaxPooling2D((2, 2), padding=self.padding, name='max_pool_'+str(0))(last_output)
        else:
            last_output = AveragePooling2D((2, 2), padding=self.padding, name='avg_pool_'+str(0))(last_output)

        for i in range(self.block_nb):

            block_input = last_output
            for j in range(self.block_size):
                last_output = Conv2D(self.filters_per_layers, self.kernel, padding=self.padding, input_shape=(32, 32, 3), name='conv_'+str(self.filters_per_layers)+'_'+str(count),
                          kernel_regularizer=self.my_reguralizer)(last_output)
                last_output = Activation(activation=self.activation, name=f"Activation_{count}")(last_output)
                if self.batch_norm == 1:
                    last_output = BatchNormalization(name='batchnorm_'+str(count))(last_output)
                count += 1
                
            
            block_input = Conv2D(self.filters_per_layers, self.kernel, padding=self.padding, input_shape=(32, 32, 3), name='conv_identity'+str(self.filters_per_layers)+'_'+str(i)
                          )(block_input)

            last_output = Add(name=f"Add_output_{i}")([last_output, block_input])
            if(i%self.filters_pool == 0):
                if self.max_or_avg_pool == 'max':
                    last_output = MaxPooling2D((2, 2), padding=self.padding, name='max_pool_'+str(count2))(last_output)
                else:
                    last_output = AveragePooling2D((2, 2), padding=self.padding, name='avg_pool_'+str(count2))(last_output)
                count2 += 1

            if(i % self.filters_double == 0):
                self.filters_per_layers = self.filters_per_layers * 2
            if self.dropout > 0:
                last_output = Dropout(self.dropout)(last_output)
            

        flatten_layer_output = Flatten(name="flatten")(last_output)  
        
        last_output = Dense(128, activation='relu', kernel_regularizer=self.my_reguralizer,
                            name='mlp_end')(flatten_layer_output)
        if self.batch_norm == 1:
                    last_output = BatchNormalization(name='batchnorm_end')(last_output)
        if self.dropout > 0:
            last_output = Dropout(self.dropout)(last_output)         
                        
        
        output_tensor = Dense(32094, activation=softmax, name=f"Dense_output")(last_output)

        return Model(input_layer, output_tensor)
    
    
        
    def train_model(self, train, verif, model):
        # Compiler le modele
        if self.optimizer == 'SGD':
            print("SGD, learning_r = ", self.learning_r, " momentum = ", self.momentum, "\n")
            opt = SGD(lr=self.learning_r, momentum=self.momentum)
        else:
            print("Adam learning_r = ", self.learning_r, " momentum = ", self.momentum, "\n")
            opt = Adam(lr=self.learning_r, beta_1=self.momentum) # beta_1 => notation pour momentum Adam
        model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        
        # Entrainer le modele
        #history = model.fit(trainX, trainY, epochs=self.epochs, batch_size=self.batch_size, 
        #                    validation_data=(testX, testY), verbose=1, callbacks=[tensorboard_callback])
        
        history = model.fit_generator(train_datagen.flow_from_dataframe(dataframe=train,
                                                      directory=DIR_TRAIN,
                                                      x_col="file_name",
                                                      y_col=["category_id"],
                                                      target_size=(64, 64),
                                                      batch_size=self.batch_size,
                                                      class_mode='raw'),
                    validation_data=train_datagen.flow_from_dataframe(
                        dataframe=verif,
                        directory=DIR_TRAIN,
                        x_col="file_name",
                        y_col=["category_id"],
                        target_size=(64, 64),
                        batch_size=self.batch_size,
                        class_mode='raw'),
                    epochs=self.epochs,
                    steps_per_epoch=len(train)//self.batch_size,
                    validation_steps=len(verif)//self.batch_size,
                    verbose=1,
                    workers=8,
                    use_multiprocessing=False)

        
        
        # Garder une trace du temps nécessaire pour fit (peut être pas la meilleure méthode)
        end = datetime.datetime.now()
        self.time_fit = end - start
        print("\nTime for fit = ", round(self.time_fit.total_seconds(),2)) # Round avec total_seconds()
        
        #Arpès que le fit soit fait, remettre filters_per_layers à sa valeur initiale pour un meilleur log 
        self.filters_per_layers = self.keep_filters_per_layers

        return history, model
    
    
    def save_model(self, history, model, main_directory, current_time):
        
        # Sauvegarde du modèle
        plot_model(model, "model.png")
        
        # Deplacement modele au bon endroit
        shutil.move(os.getcwd()+"\\model.png", main_directory+"\\log_"+self.indiv_id+"\\model.png")
        
        # Afficher nos résultats dans un graphique matplotlib sauvegardé
        pyplot.gcf().subplots_adjust(hspace = 0.5)

        # Afficher la loss
        pyplot.subplot(211)
        pyplot.title('Cross Entropy Loss')
        pyplot.plot(history.history['loss'], color='blue', label='train')
        pyplot.plot(history.history['val_loss'], color='orange', label='test')
        
        # Afficher l'accuracy
        pyplot.subplot(212)
        pyplot.title('Classification Accuracy')
        pyplot.plot(history.history['accuracy'], color='blue', label='train')
        pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
        
        # Sauvegarde
        filename = main_directory+"\\log_"+self.indiv_id+"\\"
        pyplot.savefig(filename + 'plot.png')
        pyplot.close()
       
        
        print("LOSS : ", round(history.history['loss'][self.epochs-1].item(), 3))
        print("VAL_LOSS : ", round(history.history['val_loss'][self.epochs-1].item(), 3))
        print("ACCURACY : ", round(history.history['accuracy'][self.epochs-1].item(), 3))
        print("VAL_ACCURACY : ", round(history.history['val_accuracy'][self.epochs-1].item(), 3))
        
        # attributs pour créer les csv indivudels et le csv global
        self.loss = round(history.history['loss'][self.epochs-1].item(), 3)
        self.val_loss = round(history.history['val_loss'][self.epochs-1].item(), 3)
        self.accuracy = round(history.history['accuracy'][self.epochs-1].item(), 3)
        self.val_accuracy = round(history.history['val_accuracy'][self.epochs-1].item(), 3)
        self.time_taken = round(self.time_fit.total_seconds(),2)
        
        # Créer un dataframe pandas (avec hyperparams) et le sauvegarder en CSV
        df = pd.DataFrame({'indiv_id': [self.indiv_id],
                           'epochs': [self.epochs],
                           'batch_size': [self.batch_size],
                           'block_size': [self.block_size],
                           'block_nb': [self.block_nb],
                           'l1': [self.l1],
                           'l2': [self.l2],
                           'batch_norm': [self.batch_norm],
                           'dropout': [self.dropout],
                           'filters_per_layers': [self.filters_per_layers],
                           'filters_double': [self.filters_double],
                           'filters_pool': [self.filters_pool],
                           'activation': [self.activation],
                           'kernel': [self.kernel],
                           'first_kernel': [self.first_kernel],
                           'padding': [self.padding],
                           'max_or_avg_pool': [self.max_or_avg_pool],
                           'loss': [self.loss],
                           'val_loss': [self.val_loss],
                           'accuracy': [self.accuracy],
                           'val_accuracy': [self.val_accuracy],
                           'time_taken' : [self.time_taken],
                           'learning_r' : [self.learning_r],
                           'momentum' : [self.momentum],
                           'optimizer' : [self.optimizer]
                          })
        
        df.to_csv(path_or_buf=filename+"recap.csv",index=False)
    
    # Lance toutes les étapes
    def exec_indiv(self, main_directory, current_time):
        
        # Charger les données
        #trainX, trainY, testX, testY = load_dataset()
        
        # Normaliser les données
        #trainX, testX = prep_pixels(trainX, testX)
        
        # Créer et entrainer le modele
        model = self.create_model(main_directory, shape)
        print(model.summary())
        model, model_before_train = self.train_model(train, verif, model)
        
        # Sauvegarder le modèle
        #save = self.save_model(history, model, main_directory, current_time
        
        model.save('fg_model.h5')

### Classe Python qui va démarrer les tests des neural nets


In [9]:
# Classe générale qui va nous servir à effectuer des actions sur des individus
class MyTraining:
    # Prends un ID et une liste d'individus 
    def __init__(self, id_train, indiv_list):
        
        self.id_train = id_train
        self.indiv_list = indiv_list
    
    def train(self, main_directory, current_time):
        
        print("Start training\n")
        
        for indiv in self.indiv_list:
            print("indiv ", indiv.indiv_id, "\n")
            indiv.exec_indiv(main_directory, current_time)
            print("-----------------------------------------------------------------\n")
        '''
        # Fusion des csv 
        merge_csv = pd.DataFrame(columns=['indiv_id', 'epochs', 'block_size', 'block_nb', 'l1', 'l2', 'batch_norm', 'dropout',
                                          'filters_per_layers', 'filters_double', 'filters_pool', 'activation', 'kernel', 
                                          'first_kernel', 'padding','max_or_avg_pool','loss', 'val_loss', 'accuracy', 'val_accuracy',
                                          'time_taken','learning_r', 'momentum', 'optimizer'])
        
        for indiv in self.indiv_list:
            merge_csv = merge_csv.append({'indiv_id': indiv.indiv_id, 'epochs': indiv.epochs, 'batch_size': indiv.batch_size, 'block_size' : indiv.block_size,
                                          'block_nb' : indiv.block_nb, 'l1' : indiv.l1, 'l2' : indiv.l2, 'batch_norm': indiv.batch_norm, 'dropout' : indiv.dropout,
                                          'filters_per_layers' : indiv.filters_per_layers, 'filters_double' : indiv.filters_double, 'filters_pool' : indiv.filters_pool, 
                                          'activation' : indiv.activation, 'kernel' : indiv.kernel, 'first_kernel' : indiv.first_kernel, 'padding' : indiv.padding, 
                                          'max_or_avg_pool' : indiv.max_or_avg_pool, 'loss' : indiv.loss, 'val_loss' : indiv.val_loss, 'accuracy' : indiv.accuracy, 
                                          'val_accuracy' : indiv.val_accuracy, 'time_taken' : indiv.time_taken, 'learning_r' : indiv.learning_r,
                                          'momentum': indiv.momentum, 'optimizer' : indiv.optimizer
                                         }, ignore_index=True)
        
        # sauvegarde
        merge_csv.to_csv(main_directory+"\\combined_recap.csv", index=False)
        '''    
    
    def all_indiv(self):
        
        # Affiche les caractéristiques de l'ensemble des individus
        for indiv in self.indiv_list:
            print('\n'.join(indiv.__str__()))
            for tir in range(80): print('-', end='')
            print()

### Traitement général (train de l'ensemble des modèles)

In [14]:
# Création des individus (des neurals nets, ici convnet)
list_indiv = []
for num in range(len(list_indiv_id)):
    list_indiv.append(IndividuResnetConv2d(
        list_indiv_id[num],
          list_epochs[num],
          list_batch_size[num],
          list_blocks_size[num],
          list_blocks_nb[num],
          list_l1[num],
          list_l2[num],
          list_batch_norm[num],
          list_dropout[num],
          list_filters_per_layers[num],
          list_filters_double[num],
          list_filters_pool[num],
          list_activation[num],
          list_kernel[num],
          list_first_kernel[num],
          list_padding[num],
          list_max_or_avg_pool[num],
          list_learning_r[num],
          list_momentum[num],
          list_optimizer[num]
        )
    )

# Chargement de la classe training, affichag
training_1 = MyTraining(1, list_indiv)
#training_1.all_indiv()
training_1.train(main_directory, current_time)

Start training

indiv  1 

log dir =  D:\Projets\DL_4A_2nd\HERBARIUM\utils\logs\resnetsConv2d\logs_2020-05-06-09-28\log_1\tensorboard_data\
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv_64_0 (Conv2D)              (None, 64, 64, 64)   9472        input_3[0][0]                    
__________________________________________________________________________________________________
Activation_0 (Activation)       (None, 64, 64, 64)   0           conv_64_0[0][0]                  
__________________________________________________________________________________________________
avg_pool_0 (AveragePooling2D)   (None, 32, 32, 64) 

Found 800000 validated image filenames.
Found 200000 validated image filenames.
Epoch 1/50
2500/2500 [==============================] - 204147s 82s/step - loss: 16.1180 - accuracy: 6.5000e-05 - val_loss: 16.1209 - val_accuracy: 5.0000e-05
Epoch 2/50
  78/2500 [..............................] - ETA: 54:14:23 - loss: 16.1210 - accuracy: 4.0064e-05

KeyboardInterrupt: 

### Partie tensorboard

In [ ]:
# Procedure pour utiliser tensorboard
#  1 load la première cell
#  2 utiliser la derniere cell avec --logdir (précisez bien votre répertoire, plus sur que ça
#    fonctionne avec une string "mon_path"
#  3 Vous NE POURREZ PLUS update tensorboard sur ce port et il y aura des bugs, pour éviter ça
#    quand vous voulez faire une update, fermez jupyter notebook (shutdown total) et réouvrez le 
#    OU, faites kernel->interrupt et changez de port + de folder de log

#si vous voulez tenter de tuer des process
#os.system("taskkill /im tensorboard.exe /f") #kill tous les processus qui utilisent tensorboard
#os.system('!kill 18776') #kill le processus X

In [ ]:
# Liste des ports utilisés par tensorboard, attention ça se remplit vite et il faut kill jupyter pour clean
from tensorboard import notebook
notebook.list()

In [ ]:
# Code pour démarrer tensorboard dans le dossier souhaité [PRECISEZ BIEN LE DOSSIER ICI]
%tensorboard --logdir "CONVNETS_20200119-0243\logs_20200119-093909\tensorboard_data" --port=6066

In [ ]:
# Si vous avez la folie des grandeurs
notebook.display(port=6066, height=1000) 

In [ ]:
### Fichier CSV combined_recap + Graphique

In [ ]:
# Commandes pandas utiles
data_csv = pd.read_csv(main_directory + "\\combined_recap.csv")
#data_csv = pd.read_csv("C:\\Users\\arnau\\Desktop\\quatrième_année\\Deep_Learning\\Projet_cifar-10\\CONVNETS_20200119-2043\\combined_recap.csv")
data.head()
#meilleure accuracy, moins pire loss par ex
#data_csv.sort_values(["elapsed"], axis=0, 
                 #ascending=[False], inplace=True) 

# Afficher uniquement certaines colonnes
#dataX = data_csv.filter(items=['elapsed', 'label'])

#récupérer uniquement où la loss est < à X et ou kernel = (3,3) par exemple
#dataX = data_csv.loc[(data_csv['elapsed'] > 700) & (data_csv['threadName'].str.contains('Thread Group 1-2'))]
#dataX

#pd.set_option('display.max_rows', data3.shape[0]+1) #nombre de row max à afficher
#data_csv = pd.read_csv(main_directory+"\\logs_20200116-204456\\recap.csv")
#data_csv.head()

In [ ]:
image = pyplot.imread(main_directory + "\\logs_20200119-093909\\plot.png")
#image = pyplot.imread("C:\\Users\\arnau\\Desktop\\quatrième_année\\Deep_Learning\\Projet_cifar-10\\logs_20200119-093909\\plot.png")
pyplot.imshow(image)

# PREDICTION

In [ ]:
with tf.device("/cpu:0"):
    batch_size = 32
    train_model = tf.keras.models.load_model('fg_model.h5', custom_objects={"xavier":xavier})

    test_datagen = ImageDataGenerator(featurewise_center=False,
                                      featurewise_std_normalization=False)

    generator = test_datagen.flow_from_dataframe(
            dataframe = df_images, #Limiting the test to the first 10,000 items
            directory = DIR_TEST,
            x_col = 'file_name',
            target_size=(16, 16),
            batch_size=batch_size,
            class_mode=None,  # only data, no labels
            shuffle=False)

    family, genus, category = train_model.predict_generator(generator, verbose=1)

# SUBMISSION

In [ ]:
with tf.device("/cpu:0"):
    sub = pd.DataFrame()
    sub['Id'] = df_images.id
    sub['Id'] = sub['Id'].astype('int32')

    sub['Predicted'] = np.concatenate([np.argmax(category, axis=1), 23718*np.ones((len(df_images.id)-len(category)))], axis=0)
    sub['Predicted'] = sub['Predicted'].astype('int32')
    display(sub)
    sub.to_csv('category_submission.csv', index=False)
    
    sub['Predicted'] = np.concatenate([np.argmax(family, axis=1), np.zeros((len(df_images.id)-len(family)))], axis=0)
    sub['Predicted'] = sub['Predicted'].astype('int32')
    display(sub)
    sub.to_csv('family_submission.csv', index=False)
    
    sub['Predicted'] = np.concatenate([np.argmax(genus, axis=1), np.zeros((len(df_images.id)-len(genus)))], axis=0)
    sub['Predicted'] = sub['Predicted'].astype('int32')
    display(sub)
    sub.to_csv('genus_submission.csv', index=False)
    
    
    end_time = time.time()
    total = end_time - start_time
    h = total//3600
    m = (total%3600)//60
    s = total%60
    print("Total time spent: %i hours, %i minutes, and %i seconds" %(h, m, s))